In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pathlib


In [4]:
# load a test data set got one pattern

sample_14111 = gpd.read_parquet("../cta-stop-etl/out/test/pid_14111_test_trips.parquet")

sample_95 = gpd.read_parquet("../cta-stop-etl/out/test/pid_95_test_trips.parquet")

In [8]:
sample = pd.concat([sample_14111,sample_95])
sample['time'] = sample[['bus_stop_time', 'bus_location_time']].bfill(axis=1).iloc[:, 0]
sample.drop(columns=['bus_stop_time', 'bus_location_time'], inplace=True)

In [11]:
# for now drop NA time 
sample.dropna(subset=['time'], inplace=True)
sample

,unique_trip_vehicle_day,seg_combined,typ,stpid,p_stp_id,geometry,time
4,17114111.02350517678834855510972023-01-04,14.5,B,None,None,POINT (-87.59322 41.78791),2023-01-04 06:37:00.000000000
5,17114111.02350517678834855510972023-01-04,18.0,S,13197,14111-13197,POINT (-87.59455 41.78787),2023-01-04 06:37:19.444977192
6,17114111.02350517678834855510972023-01-04,28.0,S,16036,14111-16036,POINT (-87.59808 41.79135),2023-01-04 06:39:56.887237124
7,17114111.02350517678834855510972023-01-04,30.0,S,15816,14111-15816,POINT (-87.59813 41.79485),2023-01-04 06:40:02.464339496
8,17114111.02350517678834855510972023-01-04,33.0,S,15752,14111-15752,POINT (-87.59625 41.79504),2023-01-04 06:40:23.785887686
...,...,...,...,...,...,...,...
8932,7295.02353141648835741081142023-01-03,430.5,B,NaN,None,POINT (-87.80572 41.91036),2023-01-03 14:37:00.000000000
8933,7295.02353141648835741081142023-01-03,430.5,B,NaN,None,POINT (-87.80572 41.91036),2023-01-03 14:42:00.000000000
8934,7295.02353141648835741081142023-01-03,430.5,B,NaN,None,POINT (-87.80414 41.91069),2023-01-03 14:47:00.000000000
8935,7295.02353141648835741081142023-01-03,431.0,S,NaN,95-1005,POINT (-87.80442 41.91069),2023-01-03 14:47:00.000000000


In [22]:
# how many buses pass a stop per hour by pattern
#https://pandas.pydata.org/docs/reference/series.html#datetimelike-properties

sample['hour'] = sample['time'].dt.hour
sample['day_of_year'] = sample['time'].dt.day_of_year

In [23]:
print(sample['time'].min())
print(sample['time'].max())

2023-01-03 06:32:00
2023-01-07 00:52:00


In [29]:
test_group = sample.groupby(['p_stp_id', 'day_of_year','hour']).size().reset_index(name='count')
test_group[test_group["p_stp_id"] == '95-999']

test_group.groupby(['p_stp_id', 'hour']).mean().reset_index()

,p_stp_id,hour,day_of_year,count
0,14111-10561,6,4.5,1.00
1,14111-10561,7,4.5,3.50
2,14111-10561,8,4.5,3.50
3,14111-10561,9,4.5,3.75
4,14111-10561,10,4.5,4.00
...,...,...,...,...
945,95-999,19,4.5,3.25
946,95-999,20,4.5,3.50
947,95-999,21,4.5,1.75
948,95-999,22,4.5,2.25
